In [1]:
import time
import asyncio
import pymongo
import json
from bson.json_util import dumps
from pymongo import MongoClient
from indy import anoncreds, did, ledger, pool, wallet, IndyError

async def run():
    print("Getting started -> Getting Verinym")

    # Set protocol version 2 to work with Indy Node 1.4
    await pool.set_protocol_version(2)

    pool_ = {
        'name': 'pool1',
        'config': json.dumps({"genesis_txn": '/home/indy/sandbox/pool_transactions_genesis'})
    }
    print("Open Pool Ledger: {}".format(pool_['name']))

    try:
        await pool.create_pool_ledger_config(pool_['name'], pool_['config'])
    except IndyError as ex:
        if ex.error_code == 306:
            pass
    pool_['handle'] = await pool.open_pool_ledger(pool_['name'], None)

    client = MongoClient(host='steward_db', port=27017, username='root', password='pass', authSource="admin")
    db = client["steward_db"]

    verinym_for = db.steward_tb.find_one({"name": "verinym_for"})

    entity = db.steward_tb.find_one({"name": verinym_for["value"]})

    steward = db.steward_tb.find_one({"name": "Steward"})

    await getting_verinym(steward, entity)

    db.steward_tb.find_one_and_update({"name": verinym_for["value"]}, { "$set": entity }, upsert=True);

    db.steward_tb.find_one_and_update({"name": "Steward"}, { "$set": steward }, upsert=True);

    print("==============")

    print(entity)

async def getting_verinym(from_, to):
    from_['info'] = {
        'did': to['did'],
        'verkey': to['key'],
        'role': to['role'] or None
    }
    await send_nym(from_['pool'], from_['wallet'], from_['did'], from_['info']['did'],
                   from_['info']['verkey'], from_['info']['role'])

async def send_nym(pool_handle, wallet_handle, _did, new_did, new_key, role):
    nym_request = await ledger.build_nym_request(_did, new_did, new_key, None, role)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, nym_request)


if __name__ == '__main__':
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    await run()
    time.sleep(1)  # FIXME waiting for libindy thread complete


Getting started -> Getting Verinym
Open Pool Ledger: pool1
[{"_id": {"$oid": "624ee10a00a9bf277cb9b9da"}, "name": "verinym_for", "value": "Vikram"}, {"_id": {"$oid": "624ee21577197bee8591ad5a"}, "name": "verinym_for", "value": "Vikram"}]
